This notebook create dataset config files from reference

In [53]:
import pandas as pd
import os
import glob
import json

In [45]:
## CREATE UKB40 configs for jean zay

path = "/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/UKB40/array_load/"
ref_file = f"{path}/reference.yaml"
crop_path = "/neurospin/dico/data/deep_folding/current/datasets/UkBioBank40/crops/2mm"
crop_dirs = glob.glob(f"{crop_path}/*")
crop_drops = ["CINGULATE_ALL", "ORBITAL_ALL", "CINGULATE", "deMatos_polar."]

In [20]:
## UKB40 volumes to assess regression on volume
path = "/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/UKB40/volume/"
ref_file = f"{path}/reference.yaml"
crop_path = "/neurospin/dico/data/deep_folding/current/datasets/UkBioBank40/crops/2mm"
crop_dirs = glob.glob(f"{crop_path}/*")
crop_drops = ["CINGULATE_ALL", "ORBITAL_ALL", "CINGULATE", "deMatos_polar."]

In [2]:
# create hcp configs for inference on whole dataset
path = "/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/hcp/all_data/"
ref_file = f"{path}/reference.yaml"
crop_path = "/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm"
crop_dirs = glob.glob(f"{crop_path}/*")
crop_drops = ["CINGULATE_ALL", "ORBITAL_ALL", "CINGULATE", "deMatos_polar.", "CENTRAL"]

In [3]:
# create dhcp configs for inference on whole dataset
path = "/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/dHCP/"
ref_file = f"{path}/reference.yaml"
crop_path = "/neurospin/dico/data/deep_folding/current/datasets/dHCP_374_subjects/crops/2mm"
crop_dirs = glob.glob(f"{crop_path}/*")
crop_drops = ["CINGULATE_ALL", "ORBITAL_ALL", "CINGULATE", "deMatos_polar.", "CENTRAL"]

In [63]:
# create hcp configs for pattern evaluation
# Right_FIP
path = "/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/hcp/Right_FIP"
ref_file = f"{path}/reference.yaml"
crop_path = "/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm"
crop_dirs = glob.glob(f"{crop_path}/*")
crop_drops = ["CINGULATE_ALL", "ORBITAL_ALL", "CINGULATE", "deMatos_polar.", 'CENTRAL']

In [71]:
# Left_OFC
path = "/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/hcp/Left_OFC"
ref_file = f"{path}/reference.yaml"
crop_path = "/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm"
crop_dirs = glob.glob(f"{crop_path}/*")
crop_drops = ["CINGULATE_ALL", "ORBITAL_ALL", "CINGULATE", "deMatos_polar.", 'CENTRAL']

In [79]:
# Isomap_central_left
path = "/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/hcp/Isomap_central_left"
ref_file = f"{path}/reference.yaml"
crop_path = "/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm"
crop_dirs = glob.glob(f"{crop_path}/*")
crop_drops = ["CINGULATE_ALL", "ORBITAL_ALL", "CINGULATE", "deMatos_polar.", 'CENTRAL']

In [99]:
# Right_PCS
path = "/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/ACCpatterns/Right_PCS"
ref_file = f"{path}/reference.yaml"
crop_path = "/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/crops/2mm"
crop_dirs = glob.glob(f"{crop_path}/*")
crop_drops = ["CINGULATE_ALL", "ORBITAL_ALL", "CINGULATE", "deMatos_polar.", 'CENTRAL', 'CINGULATE_FLIPPED']

In [2]:
# abide
path = "/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/abide/"
ref_file = f"{path}/reference.yaml"
crop_path = "/neurospin/dico/data/deep_folding/current/datasets/aggregate_autism/crops/2mm"
crop_dirs = glob.glob(f"{crop_path}/*")
crop_drops = ["CINGULATE_ALL", "ORBITAL_ALL", "CINGULATE", "deMatos_polar.", 'CENTRAL', 'CINGULATE_FLIPPED']

In [29]:
# bipolar
path = "/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/bipolar/"
ref_file = f"{path}/reference.yaml"
crop_path = "/neurospin/dico/data/deep_folding/current/datasets/aggregate_bipolar/crops/2mm"
crop_dirs = glob.glob(f"{crop_path}/*")
crop_drops = ["CINGULATE_ALL", "ORBITAL_ALL", "CINGULATE", "deMatos_polar.", 'CENTRAL', 'CINGULATE_FLIPPED', 'LARGE_CINGULATE.'] # TODO : process large cingulate

In [54]:
# schiz
path = "/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/schiz/"
ref_file = f"{path}/reference.yaml"
crop_path = "/neurospin/dico/data/deep_folding/current/datasets/aggregate_schizophrenia/crops/2mm"
crop_dirs = glob.glob(f"{crop_path}/*")
crop_drops = ["CINGULATE_ALL", "ORBITAL_ALL", "CINGULATE", "deMatos_polar.", 'CENTRAL', 'CINGULATE_FLIPPED', 'LARGE_CINGULATE.']

In [55]:
crop_dirs = [f for f in crop_dirs if not os.path.basename(f) in crop_drops]

In [56]:
## get the region names
ignore_regions = ['S.T.s.-S.GSM.', 'F.C.L.p.-S.GSM.', "CINGULATE_ALL", "ORBITAL_ALL", "ORBITAL", "CINGULATE", "deMatos_polar.", "deMatos."]
regions = os.listdir(crop_path)
regions = [region.replace('.','') for region in regions if region not in ignore_regions]
regions_left_right = [elem + '_left' for elem in regions] + [elem + '_right' for elem in regions]
mystring = ' '.join(regions_left_right)

In [57]:
mystring

'FColl-SRh_left SFmedian-SFpoltr-SFsup_left SFinf-BROCA-SPeCinf_left SPoC_left fronto-parietal_medial_face_left FIP_left CINGULATE_left SC-SPoC_left SFinter-SFsup_left FCMpost-SpC_left SsP-SPaint_left SOr-SOlf_left FPO-SCu-ScCal_left SFmarginal-SFinfant_left SFint-FCMant_left STi-STs-STpol_left SFint-SR_left Lobule_parietal_sup_left STi-SOTlat_left SPeC_left STsbr_left ScCal-SLi_left STs_left FCLp-subsc-FCLa-INSULA_left SC-sylv_left SC-SPeC_left OCCIPITAL_left SOr_left FColl-SRh_right SFmedian-SFpoltr-SFsup_right SFinf-BROCA-SPeCinf_right SPoC_right fronto-parietal_medial_face_right FIP_right CINGULATE_right SC-SPoC_right SFinter-SFsup_right FCMpost-SpC_right SsP-SPaint_right SOr-SOlf_right FPO-SCu-ScCal_right SFmarginal-SFinfant_right SFint-FCMant_right STi-STs-STpol_right SFint-SR_right Lobule_parietal_sup_right STi-SOTlat_right SPeC_right STsbr_right ScCal-SLi_right STs_right FCLp-subsc-FCLa-INSULA_right SC-sylv_right SC-SPeC_right OCCIPITAL_right SOr_right'

In [58]:
print(ref_file)
print('\n'.join(crop_dirs))

/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/schiz//reference.yaml
/neurospin/dico/data/deep_folding/current/datasets/aggregate_schizophrenia/crops/2mm/F.Coll.-S.Rh.
/neurospin/dico/data/deep_folding/current/datasets/aggregate_schizophrenia/crops/2mm/S.F.median-S.F.pol.tr.-S.F.sup.
/neurospin/dico/data/deep_folding/current/datasets/aggregate_schizophrenia/crops/2mm/S.F.inf.-BROCA-S.Pe.C.inf.
/neurospin/dico/data/deep_folding/current/datasets/aggregate_schizophrenia/crops/2mm/S.Po.C.
/neurospin/dico/data/deep_folding/current/datasets/aggregate_schizophrenia/crops/2mm/fronto-parietal_medial_face.
/neurospin/dico/data/deep_folding/current/datasets/aggregate_schizophrenia/crops/2mm/F.I.P.
/neurospin/dico/data/deep_folding/current/datasets/aggregate_schizophrenia/crops/2mm/S.T.s.-S.GSM.
/neurospin/dico/data/deep_folding/current/datasets/aggregate_schizophrenia/crops/2mm/CINGULATE.
/neurospin/dico/data/deep_folding/current/datasets/aggregate_s

In [59]:
# Read in the reference file
with open(ref_file, 'r') as file:
  ref = file.read()

print(ref)

# @package dataset.REPLACE_DATASET
dataset_name: REPLACE_DATASET
pickle_normal: ${dataset_folder}/aggregate_schizophrenia/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEskeleton.pkl
numpy_all: ${dataset_folder}/aggregate_schizophrenia/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEskeleton.npy
subjects_all: ${dataset_folder}/aggregate_schizophrenia/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEskeleton_subject.csv
crop_dir: ${dataset_folder}/aggregate_schizophrenia/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEcrops
foldlabel_dir: ${dataset_folder}/aggregate_schizophrenia/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDElabels
foldlabel_all: ${dataset_folder}/aggregate_schizophrenia/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDElabel.npy
subjects_foldlabel_all: ${dataset_folder}/aggregate_schizophrenia/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDElabel_subject.csv
distbottom_dir: ${dataset_folder}/aggregate_schizophrenia/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEdistbottom
distbottom_all: ${da

In [60]:
def replace_reference_yaml(crop_dir, side, ref):
  """For each crop name, it builds the yaml from the reference yaml"""
  crop_name = os.path.basename(crop_dir)
  dataset_name = crop_name.replace('.', '')
  mask_file = f"{crop_dir}/mask/{side}mask_cropped.nii.gz.minf"
  with open(mask_file, 'r') as file:
    mask = file.read()
  mask = mask.replace("attributes = ", "")
  mask = mask.replace("\'", "\"")
  # print(mask)
  mask_json = json.loads(mask)
  # print(mask_json)
  side = side
  side_long = "left" if side=='L' else "right"
  dataset_name = f"{dataset_name}_{side_long}"
  filedata = ref.replace('REPLACE_CROP_NAME', crop_name)
  filedata = filedata.replace('REPLACE_DATASET', dataset_name)
  filedata = filedata.replace('REPLACE_SIDE', side)
  filedata = filedata.replace('REPLACE_SIZEX', str(mask_json['sizeX']))
  filedata = filedata.replace('REPLACE_SIZEY', str(mask_json['sizeY']))
  filedata = filedata.replace('REPLACE_SIZEZ', str(mask_json['sizeZ']))

  result_file = f"{path}/{dataset_name}.yaml"

  return filedata, result_file

In [61]:
# Replace the target string
for crop_dir in crop_dirs:
    for side in ['L', 'R']:
        filedata, result_file = replace_reference_yaml(crop_dir, side, ref)
        print(result_file)
        with open(result_file, 'w') as file:
          file.write(filedata)
        

/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/schiz//FColl-SRh_left.yaml
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/schiz//FColl-SRh_right.yaml
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/schiz//SFmedian-SFpoltr-SFsup_left.yaml
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/schiz//SFmedian-SFpoltr-SFsup_right.yaml
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/schiz//SFinf-BROCA-SPeCinf_left.yaml
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/schiz//SFinf-BROCA-SPeCinf_right.yaml
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/schiz//SPoC_left.yaml
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/contrastive/configs/dataset/julien/schiz//SPoC_rig

In [107]:
# # Write the file out again
# with open('file.txt', 'w') as file:
#   file.write(filedata)

# Command line of train.py

In [110]:
path_champollion = "/neurospin/dico/data/deep_folding/current/models/Champollion_V0"
path_configs = path

In [124]:
trained_regions = glob.glob(f"{path_champollion}/*")
trained_regions = [f for f in trained_regions if not "README.md" in f]
trained_regions = [os.path.basename(f) for f in trained_regions]
print('\n'.join(trained_regions))

SC-sylv_left
STs-SGSM_left
SC-sylv_right
CINGULATE_right
FCLp-SGSM_left
FIP_right
ORBITAL_left


In [126]:
available_configs = glob.glob(f"{path_configs}/*")
available_configs = [f for f in available_configs if not "reference.yaml" in f]
available_configs = [os.path.basename(f).replace(".yaml", "") for f in available_configs]
print('\n'.join(available_configs))

SFint-FCMant_left
CINGULATE_right
SFint-FCMant_right
SPoC_left
STi-SOTlat_left
ScCal-SLi_left
STs_right
STs_left
SC-sylv_right
ORBITAL_left
FIP_right
SC-SPeC_left
FCMpost-SpC_right
SFinf-BROCA-SPeCinf_left
SC-SPoC_left
SC-sylv_left
ORBITAL_right
STi-SOTlat_right
SC-SPoC_right
FCLp-SGSM_left
SC-SPeC_right
STs-SGSM_left
SPoC_right
SPeC_right
SFinf-BROCA-SPeCinf_right
SPeC_left
ScCal-SLi_right
CINGULATE_left
FCMpost-SpC_left


In [134]:
to_train = set(available_configs) - set(trained_regions)
print(len(to_train))
to_train = [f"with_reskel_distbottom/2mm/UKB/{i}" for i in to_train]

22


In [133]:
','.join(to_train)

'with_reskel_distbottom/2mm/UKB/STs_left,with_reskel_distbottom/2mm/UKB/SPoC_left,with_reskel_distbottom/2mm/UKB/STi-SOTlat_left,with_reskel_distbottom/2mm/UKB/STi-SOTlat_right,with_reskel_distbottom/2mm/UKB/SC-SPoC_left,with_reskel_distbottom/2mm/UKB/SFint-FCMant_left,with_reskel_distbottom/2mm/UKB/SPoC_right,with_reskel_distbottom/2mm/UKB/SC-SPeC_left,with_reskel_distbottom/2mm/UKB/ScCal-SLi_left,with_reskel_distbottom/2mm/UKB/STs_right,with_reskel_distbottom/2mm/UKB/SPeC_left,with_reskel_distbottom/2mm/UKB/SPeC_right,with_reskel_distbottom/2mm/UKB/ORBITAL_right,with_reskel_distbottom/2mm/UKB/SFint-FCMant_right,with_reskel_distbottom/2mm/UKB/SC-SPoC_right,with_reskel_distbottom/2mm/UKB/FCMpost-SpC_left,with_reskel_distbottom/2mm/UKB/FCMpost-SpC_right,with_reskel_distbottom/2mm/UKB/SFinf-BROCA-SPeCinf_right,with_reskel_distbottom/2mm/UKB/ScCal-SLi_right,with_reskel_distbottom/2mm/UKB/SFinf-BROCA-SPeCinf_left,with_reskel_distbottom/2mm/UKB/CINGULATE_left,with_reskel_distbottom/2mm/UKB/

In [135]:
len(to_train)

22